# Answers of the problem set 2 in python

In [33]:
import numpy as np
from scipy.stats import norm

In [38]:
# Question 3

#creating variables 
beta = 0.987 # intertemporal discount rate (discrete time)

mu = 2 # risk aversion

alpha = 1/3 # income of labor/ toal income 

delta = 0.012 # depreciation rate 

rho = 0.95 # persistence of the previous shock

sigma = 0.007 # variance of the white error of the shock

tol = 10^(-5)

k_ss = (1/(alpha*beta) - (1-delta)/alpha)**(1/(alpha-1))


In [ ]:
# Tauchen Function
N = 7 # number of grid points
m = 3 # number of standard deviations

# z_grid is schock grid, pi is transition matrix
def tauchen(rho, sigma, m, N):
    sigma_z = sigma/((1-rho**2)**0.5)
    z_grid = np.zeros(N)
    z_grid[0] = -m*sigma_z
    z_grid[N-1] = m*sigma_z
    z_grid[1:N-1] = np.linspace(z_grid[0], z_grid[N-1], N-2)
    pi = np.zeros((N, N))
    for k_idx in range(N):
        for z_idx in range(N):
            if z_idx == 0:
                pi[k_idx, z_idx] = norm.cdf((z_grid[0] - rho*z_grid[k_idx] + (z_grid[1]-z_grid[0])/2)/sigma)
            elif z_idx == N-1:
                pi[k_idx, z_idx] = 1 - norm.cdf((z_grid[N-1] - rho*z_grid[k_idx] - (z_grid[N-1]-z_grid[N-2])/2)/sigma)
            else:
                pi[k_idx, z_idx] = norm.cdf((z_grid[z_idx] - rho*z_grid[k_idx] + (z_grid[1]-z_grid[0])/2)/sigma) - norm.cdf((z_grid[z_idx] - rho*z_grid[k_idx] - (z_grid[z_idx]-z_grid[z_idx-1])/2)/sigma)
    return z_grid, pi

z_grid, pi = tauchen(rho, sigma, m, N)


array([[3.15477021e-01, 0.00000000e+00, 2.73242879e-02, 8.75655104e-12,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.15477021e-01, 0.00000000e+00, 2.73242879e-02, 8.75655104e-12,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.27905735e-07, 0.00000000e+00, 4.00971109e-01, 1.53169804e-02,
        1.63535852e-12, 0.00000000e+00, 0.00000000e+00],
       [3.70976716e-22, 0.00000000e+00, 7.78237899e-07, 4.91845414e-01,
        8.15380770e-03, 2.88546964e-13, 0.00000000e+00],
       [6.89208451e-46, 0.00000000e+00, 3.71364638e-21, 2.51356290e-06,
        5.79589888e-01, 4.11928151e-03, 2.27905735e-07],
       [1.28335356e-78, 0.00000000e+00, 2.04854313e-44, 3.51129030e-20,
        7.68003583e-06, 6.57191011e-01, 3.15477021e-01],
       [1.28335356e-78, 0.00000000e+00, 2.04854313e-44, 3.51129030e-20,
        7.68003583e-06, 6.57191011e-01, 3.15477021e-01]])

In [44]:
# Iteration of the fucntion value 
k_grid_size = 500
k_grid = np.linspace(0.75*k_ss, 1.25*k_ss, k_grid_size) # create a grid for capital

iter = 0
max_iter = 10_000

V = np.zeros((k_grid_size, N)) # The value function matrix starts with zeros
V_new = np.zeros((k_grid_size, N)) # The new value function matrix starts with zeros

policy_k_prime = np.zeros((k_grid_size, N)) # The policy function matrix starts with zeros

error_V_Vnew = tol + 1 # Initialize the error of the difference between the value function and the new value function

# The utility function
def u(c, mu):
    if c<0:
        return -1000000000
    else:
        return (c**(1-mu)-1)/(1-mu)






In [ ]:
#Brute Force Iteration:
while iter < max_iter and error_V_Vnew > tol:
    for k_idx in range(k_grid_size):
        for z_idx in range(N):
            V_new[k_idx, z_idx] = -1000000000
            for k_prime in range(k_grid_size):
                c = (1-delta)*k_grid[k_idx] + z_grid[z_idx]*k_grid[k_idx]**alpha - k_grid[k_prime]
                if c>0: # Am i right to create this condition? Or should  i call the utility function and set the utility function to -1000000000 if c<0?
                    V_temp = u(c, mu) + beta*np.dot(V[k_prime, :], pi[z_idx, :])
                    if V_temp > V_new[k_idx, z_idx]:
                        V_new[k_idx, z_idx] = V_temp
                        policy_k_prime[k_idx, z_idx] = k_grid[k_prime]
    error_V_Vnew = np.max(np.abs(V - V_new))
    V = np.copy(V_new)
    iter += 1